# <3장 : 고양이와 개 : 30줄의 케라스로 구현한 전이 학습>

# 1. 전이 학습 (Transfer Train)


"미리학습된 공통적인 계층들은 활용하고, 특정 분류 작업에 적합한 계층들을 별도로 추가 및 제거하여 학습하는 방법"

일반적으로 새로운 분류기 모델을 만들기 위해 공통계층 뒤에 2~3개의 FC layer들을 추가

# 2. 미세 조정 (Fine Tune)

"사전 학습된 모델에서 하위계층의 가중치는 그대로 사용하고, 기존 작업과 관련된 상위 계층을 제거하고 새로운 계층으루 추가하여 훈련하는 것"

전이 학습과 미세 조정은 서로 교환적으로 사용되며, 전이 학습은 일반적인 개념으로 많이 사용되고, 미세 조정은 구현이라는 의미로 사용됨

훈련할 데이터 셋이 많은 양을 보유한 경우, 모든 레이어에 대한 미세 조정을 수행하고

훈련할 데이터 셋의 양이 적을 경우, 마지막 몇 개의 레이어들에 대해서만 미세 조정을 수행함

# 3. 전이 학습과 케라스를 이용한 사용자 정의 분류기 만들기

Kaggle에서 개와 고양이로 구성된 데이터셋 로드, 

로드된 데이터셋을 Train_set과 Validation_set으로 분리하여 적절한 폴더에 저장

리눅스의 경우 아래의 코드를 터미널에 입력하여 진행

윈도우의 경우 다음 링크로 들어가 다운로드 진행 (윈도우는 다음과 같이 진행이 안됨)

https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

In [3]:
!wget https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/trani.zip
!unzip train.zip
!mv train data
!cd data
!mkdir train val
!mkdir train/cat train/dog
!mkdir val/cat val/dog
!ls | grep cat | sort -R | head -250 | xargs -I my train/cat
!ls | grep dog | sort -R | head -250 | xargs -I my train/dog
!ls | grep cat | sort -R | head -250 | xargs -I my val/cat
!ls | grep dog | sort -R | head -250 | xargs -I my val/dog

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'unzip'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'mv'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
지정된 경로를 찾을 수 없습니다.
하위 디렉터리 또는 파일 train이(가) 이미 있습니다.
다음 내용 진행 중 오류 발생: train.
하위 디렉터리 또는 파일 val이(가) 이미 있습니다.
다음 내용 진행 중 오류 발생: val.
명령 구문이 올바르지 않습니다.
명령 구문이 올바르지 않습니다.
'ls'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'ls'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'ls'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'ls'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


# 데이터 파이프 라인 구축

In [73]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math

In [83]:
TRAIN_DATA_DIR = "../sugo3/OneDrive/바탕 화면/Kaggle Dataset/cat and dog/train/"
VALIDATION_DATA_DIR = "../sugo3/OneDrive/바탕 화면/Kaggle Dataset/cat and dog/val/"
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2 # 분류할 클래스의 수
IMG_WIDTH, IMG_HEIGHT = 224, 224 # 이미지 입력을 위한 이미지 크기 조정
BATCH_SIZE = 64 # 학습에 사용할 데이터들의 배치 크기

In [84]:
train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input, # 픽셀들의 정규롸를 위해 preprocess_input을 사용
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=-0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [85]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=12345, # 난수 생성하기 위한 seed
    class_mode = 'categorical'
)

val_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=12345,
    class_mode = 'categorical'
)

Found 20001 images belonging to 2 classes.
Found 4999 images belonging to 2 classes.


# 모델 정의

In [89]:
def model_maker():
    base_model = MobileNet(include_top=False, input_shape = (IMG_WIDTH, IMG_HEIGHT,3))
    for layer in base_model.layers[:]:
        layer.trainable = False # Freeze the layers
    inputs = Input(shape=(IMG_WIDTH, IMG_HEIGHT,3))
    custom_model = base_model(inputs)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation = 'relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation = 'softmax')(custom_model)
    return Model(inputs = inputs, outputs = predictions)

# 모델 훈련
1. 훈련 매개 변수 설정

In [90]:
model = model_maker()

In [93]:
model.compile(loss = 'binary_crossentropy',
             optimizer = tf.keras.optimizers.Adam(lr=0.001),
             metrics = ['acc'])
model.fit_generator(train_generator,
                   steps_per_epoch=math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE),
                    epochs = 10,
                    validation_data=val_generator,
                    validation_steps=math.ceil(float(VALIDATION_SAMPLES)/BATCH_SIZE))

Epoch 1/10
8/8 [==============================] - 12s 944ms/step - loss: 0.6388 - acc: 0.7039 - val_loss: 0.2210 - val_acc: 0.9570
Epoch 2/10
8/8 [==============================] - 7s 950ms/step - loss: 0.2827 - acc: 0.9092 - val_loss: 0.1126 - val_acc: 0.9648
Epoch 3/10
8/8 [==============================] - 7s 932ms/step - loss: 0.1991 - acc: 0.9343 - val_loss: 0.0712 - val_acc: 0.9824
Epoch 4/10
8/8 [==============================] - 7s 939ms/step - loss: 0.1454 - acc: 0.9560 - val_loss: 0.1017 - val_acc: 0.9629
Epoch 5/10
8/8 [==============================] - 7s 929ms/step - loss: 0.1728 - acc: 0.9495 - val_loss: 0.0646 - val_acc: 0.9766
Epoch 6/10
8/8 [==============================] - 7s 934ms/step - loss: 0.1788 - acc: 0.9396 - val_loss: 0.0650 - val_acc: 0.9707
Epoch 7/10
8/8 [==============================] - 7s 930ms/step - loss: 0.0809 - acc: 0.9815 - val_loss: 0.0845 - val_acc: 0.9707
Epoch 8/10
8/8 [==============================] - 7s 941ms/step - loss: 0.1632 - acc: 0.9